# Setup GitHub actions to Build Docker image automatically

As part of having a continuous integration workflow GitHub actions is utilized to rebuild Docker images whenever new code is pushed to a specific directory.

In order to utilize GitHub actions a directory named ***.github*** needs to be created in the base directory of the repository.
The ***.github*** directory should also contain a directory named ***workflows*** in order to store and run the different [workflows](https://docs.github.com/en/actions/using-workflows/about-workflows) created.
The workflow itself is a [***yaml***](https://docs.github.com/en/actions/using-workflows/workflow-syntax-for-github-actions) file that defines the actions that [trigger](https://docs.github.com/en/actions/using-workflows/triggering-a-workflow) and execute what is required for the continuous integration application. 

## Example ***.github/workflows/build-push-basenb.yaml***

```{note}
There is an opportunity to utilize self-hosted runners for GitHub actions as well. The cisl-cloud repository is setup to run workflows directly on the on-premise hardware. Directions on how to this can be configured for other repositories can be viewed at this [link to the self-hosted runner documentation](https://ncar.github.io/cisl-cloud/how-to/github-actions.html#self-hosted-runners)
```

This example workflow utilizes a few different github actions to build a docker image and push it to a container registry. Each step is explained in detail via inline comments. 

```
# This workflow builds a CPU JupyterHub single user container image and pushes it to the NSF NCAR CISL Harbor Repository
# This workflow is specific to the base-notebook directory and image
# Set the workflow name
name: JHub CPU Build & Push

# Define the trigger that starts the action
# For this workflow the trigger is on a push that changes anything in the configs/jupyter/base-notebook/ path on the main branch
on:
  push:
    paths:
      - configs/jupyter/base-notebook/**
    branches:
      - main

# Define the actions that are going to take place as part of this workflow    
jobs:
  # Name the job(s)
  podman-build-push-base-notebook:
    # Define where the job should run
    # This repository has a self-hosted runner configured and uses on-premise cloud resources for the job
    runs-on: self-hosted
    # Set the steps to take in order
    steps:
      # Step 1 is to checkout the github repo used to build the Dockerfile
      - name: Check out the repo
        uses: actions/checkout@v4
      # Get the date to apply to image tag
      - name: Get current date
        id: date
        run: echo "date=$(date +'%Y-%m-%d.%H')" >> $GITHUB_OUTPUT
      - name: Build Podman image
        run: |
          podman build -f configs/jupyter/gpu-pyt-notebook/Dockerfile -t hub.k8s.ucar.edu/cislcloudpilot/cisl-cloud-base:${{ steps.date.outputs.date }} configs/jupyter/gpu-pyt-notebook/. --no-cache --format=docker
      - name: Login to Harbor
        run: podman login hub.k8s.ucar.edu -u Harbor -p ${{ secrets.HARBOR_LOGIN  }}
      - name: Push Podman image to Docker hub
        run: |
          podman push hub.k8s.ucar.edu/cislcloudpilot/cisl-cloud-base:${{ steps.date.outputs.date }}
```